Running ES 8.7.16

curl localhost:9200

"42f05b9372a9a4a470db3b52817899b99a76ee73"

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Indexing Docs

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch('http://localhost:9200')

In [4]:
index_settings = {
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{
        "properties":{
            "text": {"type":"text"},
            "section": {"type":"text"},
            "question": {"type":"text"},
            "course": {"type":"keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/RD2fzl3YRmescZLtmp8zZg] already exists')

In [5]:
from tqdm.auto import tqdm

In [6]:
for doc in tqdm(documents):    
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Executing query

In [19]:
query = "How do copy a file to a Docker container?"

In [40]:
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": { 
                    "multi_match": { #Text matching
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

In [41]:
response = es_client.search(index=index_name, body=search_query)

In [42]:
response

ObjectApiResponse({'took': 14, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 680, 'relation': 'eq'}, 'max_score': 73.50458, 'hits': [{'_index': 'course-questions', '_id': 'qog1MZoBdJPdbTXLBy-E', '_score': 73.50458, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'XohAMZoBdJPdbTXLCjNV', '_score': 73.50458, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run

44.5


In [23]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [24]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from

How do I copy files from a different folder into docker container’s working directory?

### Building Prompt

In [33]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [34]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [43]:
context_parts = []

for hit in response['hits']['hits']:
    doc = hit['_source']
    context_part = context_template.format(**doc)
    context_parts.append(context_part)

context = '\n\n'.join(context_parts)

prompt = prompt_template.format(question=query, context=context)

In [44]:
len(prompt)

1383

In [39]:
print(prompt[:100])

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.



### Tokens

In [45]:
import tiktoken

In [52]:
encoding = tiktoken.encoding_for_model("gpt-4o")


In [50]:
len(encoding.encode(prompt))

308

### Generating answer

In [53]:
from huggingface_hub import InferenceClient

In [54]:
client = InferenceClient()

In [57]:
response = client.chat_completion(
    model= "meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages = [{"role": "user", "content": prompt}]
)

In [58]:
response.choices[0].message.content

"To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows:\n\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n\nFor example, if you want to copy a file named 'example.txt' from your local machine to a container with the ID 'abc123' in the 'Documents' directory, you would use:\n\ndocker cp /path/to/local/example.txt abc123:/Documents"